In [2]:
# set autoreload
%load_ext autoreload
%autoreload 1
%aimport cgf_utils
%aimport mf
%aimport paths
%aimport income_funcs

import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import rasterio as rio
import rasterra as rt
from pathlib import Path
import geopandas as gpd
from pymer4 import Lmer
import glob
import logging
import pickle
import sys
import logging
from scipy.special import expit

from location_mapping import load_fhs_lsae_mapping
#from income_funcs import load_binned_income_distribution_proportions


In [2]:
GLOBAL_POPULATION_FILEPATH = '/mnt/team/rapidresponse/pub/population/data/01-raw-data/other-gridded-pop-projects/global-human-settlement-layer/2020/GHS_POP_E2020_GLOBE_R2023A_4326_30ss_V1_0.tif'

In [30]:
measure = 'stunting'
fhs_location_id = 573 #98 #Chile
scenario = 'ssp119'
year_id = 2022
sex_id = 2
age_group_id = 4
model_identifier = 'tempgrid_o30'#'re_grid_o30'
#model_filepath = paths.MODEL_ROOTS / model_identifier / f'model_{measure}_{age_group_id}_{sex_id}.pkl'

# model_filepath = '/mnt/team/rapidresponse/pub/population/modeling/climate_malnutrition/output/models/model.pkl'
# with open(model_filepath, 'rb') as f:
#     model = pickle.load(f)

In [35]:
%aimport fhs_prediction
# get_predictions(measure, fhs_location_id, scenario, year, model_identifier):

fhs_prediction.get_predictions(measure, fhs_location_id, scenario, year_id, model_identifier)

Loading /mnt/team/rapidresponse/pub/population/modeling/climate_malnutrition/output/models/tempgrid_o30/model_stunting_5_1.pkl
Loading /mnt/team/rapidresponse/pub/population/modeling/climate_malnutrition/output/models/tempgrid_o30/model_stunting_4_1.pkl
Loading /mnt/team/rapidresponse/pub/population/modeling/climate_malnutrition/output/models/tempgrid_o30/model_stunting_4_2.pkl
Loading /mnt/team/rapidresponse/pub/population/modeling/climate_malnutrition/output/models/tempgrid_o30/model_stunting_5_2.pkl


affected_proportion  ...   measure
fhs_location_id age_group_id sex_id                       ...          
573.0           4            1                  0.293676  ...  stunting
                             2                  0.217817  ...  stunting
                5            1                  0.428143  ...  stunting
                             2                  0.377182  ...  stunting

[4 rows x 4 columns]

In [24]:
fhs_location_id

171

In [12]:
def get_model_family(model_identifier, measure):
    import glob
    import re
    folder_path = paths.MODEL_ROOTS / model_identifier 
    glob_pattern = f"{folder_path}/model_*_*.pkl"
    pattern = re.compile(f"model_{measure}_" + r"(\d+)_(\d+)\.pkl")
    models = []

    for filepath in folder_path.glob("model_*_*.pkl"):
        match = pattern.match(filepath.name) 
        if match:
            # Extract X and Y from the filename
            age_group_id, sex_id = match.groups()
            with open(filepath, 'rb') as f:
                models.append({'model': pickle.load(f), 'age_group_id': age_group_id, 'sex_id': sex_id})
        else:
            raise ValueError(f"Filename {filename} does not match the pattern {pattern.pattern}")
    return model_spec

def scale_like_input_data(to_scale, input_min, input_max):
    return (to_scale - input_min) / (input_max - input_min)

def reverse_scaling(X_scaled, original_min, original_max, scaled_min, scaled_max):
    X_std = (X_scaled - scaled_min) / (scaled_max - scaled_min)
    X_original = X_std * (original_max - original_min) + original_min
    return X_original


NameError: name 'model' is not defined

In [6]:
loc_mapping = load_fhs_lsae_mapping(fhs_location_id)
fhs_shapefile = loc_mapping.iloc[0].fhs_shape
location_iso3 = loc_mapping.iloc[0].worldpop_iso3
simple_loc_mapping = loc_mapping[['fhs_location_id', 'lsae_location_id']]
income_df = income_funcs.load_binned_income_distribution_proportions(fhs_location_id=fhs_location_id, measure= measure, year_id = year_id) #and year

models = get_model_family(model_identifier, measure)
model = models[0]['model'] #getting the first model to base bins and variable info off of

In [17]:
fhs_pop_raster = rt.load_raster(GLOBAL_POPULATION_FILEPATH, fhs_shapefile.bounds).set_no_data_value(np.nan)
fhs_pop_raster = fhs_pop_raster.clip(fhs_shapefile)

possible_climate_variables = ['temp', 'precip', 'over_30']
climate_vars_to_match_bins = [v for v in model.vars_to_bin if v in possible_climate_variables]
continuous_climate_vars = [v for v in ['temp', 'over_30'] if (v in possible_climate_variables) and (v not in climate_vars_to_match_bins) and (v in model.model_vars)]
climate_vars = [x for x in possible_climate_variables if x in set(model.model_vars + model.vars_to_bin)]

climate_rasters = {}
for var in climate_vars:
    climate_rasters[var] = mf.get_climate_variable_raster(scenario, year_id, var, None, None, untreated=True)
    climate_rasters[var] = climate_rasters[var].resample_to(fhs_pop_raster).clip(fhs_shapefile)
    


In [39]:
#def get_climate_variable_raster(location_iso3, scenario, year, climate_var, shapefile, reference_raster, nodata = np.nan, untreated=False):

admin_dfs = []
for _, admin2_row in loc_mapping.iterrows():
    lsae_location_id = admin2_row.lsae_location_id
    lsae_shapefile = admin2_row.lsae_shape

    pop_raster = fhs_pop_raster.clip(lsae_shapefile).mask(lsae_shapefile)
    pop_array = pop_raster.set_no_data_value(np.nan).to_numpy()

    rasters = {'population': pop_array.flatten()}

    for var in climate_vars_to_match_bins:
        climate_raster = climate_rasters[var].clip(lsae_shapefile).mask(lsae_shapefile)
        climate_array = climate_raster.to_numpy()
        assert pop_array.shape == climate_array.shape
        binned_climate_array = np.digitize(climate_array, model.var_info[var]['bin_edges'], right=False) - 1
        rasters[var+'_bin_idx'] = binned_climate_array.flatten()

    #climate_raster = big_climate_raster.clip(lsae_shapefile).mask(lsae_shapefile)#.resample_to(pop_raster)
    for var in continuous_climate_vars:
        climate_raster = climate_rasters[var].clip(lsae_shapefile).mask(lsae_shapefile)
        climate_array = climate_raster.to_numpy()
        assert pop_array.shape == climate_array.shape
        rasters[var] = climate_array.flatten()

    # Alternative approach is to group pixels to lsae
    #temp_df = pd.DataFrame({'pop': pop_array.flatten(), 'climate_bin_idx': binned_climate_array.flatten()}).groupby('climate_bin_idx', as_index=False).pop.sum()
    # Keeping it as pixels
    pixels_df = pd.DataFrame(rasters)
    pixels_df = pixels_df.dropna(subset=['population'])
    pixels_df['lsae_pop'] = np.nansum(pop_array)
    pixels_df['lsae_location_id'] = lsae_location_id
    pixels_df['worldpop_iso3'] = admin2_row.worldpop_iso3

    local_income_df = income_df.query('lsae_location_id == @lsae_location_id')
    for var in climate_vars_to_match_bins:
        pixels_df = pixels_df.merge(model.var_info[var]['bins'], left_on=var+'_bin_idx', right_index=True, how='inner')
    pixels_df = pixels_df.merge(local_income_df, on='lsae_location_id', how='left')

    # The lines from now on have coefficients and so are age_group and sex_id - specific
    # Parallelizing by them could happen here - These can be precomputed for a model
    for m in models:
        model = m['model']
        px_for_model_df = pixels_df.copy()
        model.var_info['ihme_loc_id']['coefs'].rename(columns={'ihme_loc_id': 'worldpop_iso3'}, inplace=True)
        
        if model.has_grid:
            px_for_model_df = px_for_model_df.merge(model.grid_spec['grid_definition'], how='left')
            px_for_model_df = px_for_model_df.merge(model.var_info['grid_cell']['coefs'], how='left')

        for var in continuous_climate_vars:
            assert(len(model.var_info[var]['coefs'].columns) == 1)
            colname = model.var_info[var]['coefs'].columns[0]
            px_for_model_df[colname] = model.var_info[var]['coefs'][colname].item()

        px_for_model_df = px_for_model_df.merge(model.var_info['ihme_loc_id']['coefs'], how='left', on='worldpop_iso3') 
        px_for_model_df.ihme_loc_id_coef = px_for_model_df.ihme_loc_id_coef.fillna(0)
        
        #build the logistic input one variable at a time
        px_for_model_df['logistic_input'] = model.var_info['intercept']['coef']
        px_for_model_df['logistic_input'] += px_for_model_df['ihme_loc_id_coef']
        for var in climate_vars_to_match_bins:
            if var in model.grid_spec['grid_order']: continue
            px_for_model_df['logistic_input'] += px_for_model_df[var+'_bin_coef']
        for var in continuous_climate_vars:
            px_for_model_df['logistic_input'] += px_for_model_df[var+'_coef']
        if model.has_grid:
            px_for_model_df['logistic_input'] += px_for_model_df['grid_cell_coef']
        px_for_model_df['prediction'] = expit(px_for_model_df['logistic_input'])
        px_for_model_df['age_group_id'] = m['age_group_id']
        px_for_model_df['sex_id'] = m['sex_id']
        admin_dfs.append(px_for_model_df)

fhs_df = pd.concat(admin_dfs)
#24s to 92s
#2:27 to 

In [41]:
fhs_df['population_at_income'] = fhs_df['population'] * fhs_df['proportion_at_income']
fhs_df['population_proportion_at_income'] = fhs_df['population_at_income'] / fhs_df['lsae_pop'] / len(loc_mapping)
fhs_df['affected_proportion'] = fhs_df['population_proportion_at_income'] * fhs_df['prediction']

result_df = fhs_df.groupby(['fhs_location_id', 'age_group_id', 'sex_id']).agg(susceptible_proportion = ('population_proportion_at_income', 'sum'), affected_proportion = ('affected_proportion', 'sum'))
result_df['year_id'] = year_id
result_df['scenario'] = scenario
result_df['measure'] = measure

return result_df

#return result
